In [27]:
### ------------ ###
### System Setup ###
### ------------ ###

# Import modules/methods
import pandas as pd

### ------------ ###
### Pipeline dev ###
### ------------ ###

# Read in data
redfin = pd.read_csv('https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/county_market_tracker.tsv000.gz', sep = '\t')
re_inv = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv')
re_hot = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Hotness/RDC_Inventory_Hotness_Metrics_County_History.csv')

# Drop last rows of Realtor.com data (junk lines; no real data)
re_inv.drop(re_inv.tail(1).index, inplace=True)
re_hot.drop(re_hot.tail(1).index, inplace=True)

# Fields to keep from Redfin
keep_redfin = ['period_begin', 'region', 'property_type', 'median_list_price', 'median_sale_price'
                , 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'avg_sale_to_list', 'price_drops', 'sold_above_list', 'median_dom']

# Fields to keep from Realtor.com inventory data
keep_re_inv = ['month_date_yyyymm', 'county_fips', 'county_name', 'median_listing_price', 'pending_listing_count', 'new_listing_count', 'total_listing_count'
                , 'price_reduced_count', 'median_days_on_market']

# Fields to keep from Realtor.com hotness data
keep_re_hot = ['month_date_yyyymm', 'county_fips', 'county_name', 'hotness_rank', 'hotness_score', 'supply_score'
                , 'demand_score', 'ldp_unique_viewers_per_property_mm']

# Drop unwanted fields
redfin = redfin[keep_redfin]
re_inv = re_inv[keep_re_inv]
re_hot = re_hot[keep_re_hot]

# Rename fields
rename_redfin = ['year_month', 'county_name', 'property_type', 'median_list_price', 'median_sale_price', 'homes_sold', 'pending_sales', 'new_listings'
                , 'total_inventory', 'avg_sale_to_list', 'price_drops', 'sold_above_list', 'median_dom']
rename_re_inv = ['year_month', 'county_fips', 'county_name', 'median_list_price', 'pending_sales', 'new_listings', 'total_inventory', 'price_drops', 'median_dom']
rename_re_hot = ['year_month', 'county_fips', 'county_name', 'hotness_rank', 'hotness_score', 'supply_score', 'demand_score', 'unq_viewers_mm']

redfin.columns = rename_redfin
re_inv.columns = rename_re_inv
re_hot.columns = rename_re_hot

# Clean up text fields
redfin['county_name'] = redfin['county_name'].str.upper()
re_inv['county_name'] = re_inv['county_name'].str.upper()
re_hot['county_name'] = re_hot['county_name'].str.upper()

redfin['property_type'] = redfin['property_type'].str.upper()

# Convert data types
redfin['year_month'] = pd.to_datetime(redfin['year_month'])
redfin['county_name'] = redfin['county_name'].astype('str')
redfin['property_type'] = redfin['property_type'].astype('str')

re_inv['county_fips'] = re_inv['county_fips'].astype('int')
re_hot['county_fips'] = re_hot['county_fips'].astype('int')

re_inv['year_month'] = pd.to_datetime(re_inv['year_month'].astype('str').str.slice(0,4) + '-' + re_inv['year_month'].astype('str').str.slice(4,6) + '-01')
re_hot['year_month'] = pd.to_datetime(re_hot['year_month'].astype('str').str.slice(0,4) + '-' + re_hot['year_month'].astype('str').str.slice(4,6) + '-01')

# Merge Realtor.com data
realtor = pd.merge(re_inv, re_hot, on = ['year_month', 'county_name', 'county_fips'], how = 'left')

# Convert to Parquet
redfin.to_parquet('../data/redfin.parquet')
realtor.to_parquet('../data/realtor.parquet')

/var/folders/27/22xv7cnn5dv21ywhxhvgv7p40000gn/T/ipykernel_30330/3154758433.py:14: DtypeWarning: Columns (0,1,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  re_inv = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv')
/var/folders/27/22xv7cnn5dv21ywhxhvgv7p40000gn/T/ipykernel_30330/3154758433.py:15: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  re_hot = pd.read_csv('https://econdata.s3-us-west-2.amazonaws.com/Reports/Hotness/RDC_Inventory_Hotness_Metrics_County_History.csv')
/var/folders/27/22xv7cnn5dv21ywhxhvgv7p40000gn/T/ipykernel_30330/3154758433.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

,year_month,county_fips,county_name,median_list_price,pending_sales,new_listings,total_inventory,price_drops,median_dom,hotness_rank,hotness_score,supply_score,demand_score,unq_viewers_mm
883,2022-03-01,47037,"DAVIDSON, TN",499900.0,760.0,944.0,1247.0,112.0,11.0,214.0,75.554851,98.985415,52.124287,-0.079140
3287,2022-02-01,47037,"DAVIDSON, TN",499450.0,775.0,820.0,1217.0,80.0,12.0,179.0,78.503488,99.556119,57.450856,0.046691
8075,2022-01-01,47037,"DAVIDSON, TN",475900.0,749.0,744.0,1222.0,84.0,32.0,151.0,80.405834,97.463538,63.348129,0.608300
11191,2021-12-01,47037,"DAVIDSON, TN",471400.0,895.0,716.0,1533.0,148.0,29.0,227.0,74.476855,98.858592,50.095117,0.034196
13515,2021-11-01,47037,"DAVIDSON, TN",456995.0,1008.0,1032.0,1842.0,256.0,27.0,338.0,68.325935,96.575777,40.076094,0.000421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200320,2016-11-01,47037,"DAVIDSON, TN",347450.0,2.0,1180.0,3008.0,676.0,43.0,NaN,NaN,NaN,NaN,NaN
202585,2016-10-01,47037,"DAVIDSON, TN",334900.0,2.0,1248.0,3238.0,784.0,38.0,NaN,NaN,NaN,NaN,NaN
207493,2016-09-01,47037,"DAVIDSON, TN",329950.0,2.0,1320.0,3221.0,700.0,37.0,NaN,NaN,NaN,NaN,NaN
210220,2016-08-01,47037,"DAVIDSON, TN",324950.0,2.0,1452.0,3198.0,624.0,37.0,NaN,NaN,NaN,NaN,NaN
